In [1]:
!pip install ../input/huggingface-transformers/sacremoses-master/sacremoses-master
!pip install ../input/huggingface-transformers/transformers-master/transformers-master

Processing /kaggle/input/huggingface-transformers/sacremoses-master/sacremoses-master
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=882724 sha256=25ec6bafb72a99e2312141c6f1de8e672ecbaacd28415efeb16bbcfdf5155c18
  Stored in directory: /tmp/.cache/pip/wheels/b3/c1/90/fb74570cc1a23673f29766b39fa295da523fac9813fb3f32c9
Successfully built sacremoses
Processing /kaggle/input/huggingface-transformers/transformers-master/transformers-master
  Created wheel for transformers: filename=transformers-2.2.2-cp36-none-any.whl size=415676 sha256=550cfa8d1a7521275e974f9149c9aaed12345745c844926ba992219e67d6f45c
  Stored in directory: /tmp/.cache/pip/wheels/7e/f5/40/5ef4fd8956cee49677c0bdfe569e8aa2cc7f2441e8bef98d1a
Successfully built transformers


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import gc
from sklearn.model_selection import train_test_split, GroupKFold
import bert_tokenization as tokenization
from scipy.stats import spearmanr
from math import floor, ceil
from transformers import TFPreTrainedModel, TFBertPreTrainedModel, TFBertMainLayer, BertConfig, TFBertModel, BertTokenizer, TFBertForSequenceClassification
from transformers.modeling_tf_utils import get_initializer
from tqdm.notebook import tqdm
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import pyprind 
import warnings
warnings.filterwarnings('ignore')
import operator
import string
import json
import re
import gensim
from gensim.models import KeyedVectors
import seaborn as sns
from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()

# import pytorch_transformers

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# tf.executing_eagerly()
import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [3]:
PATH = '../input/google-quest-challenge/'
BERT_PATH = '../input/bert-base-from-tfhub/bert_en_uncased_L-12_H-768_A-12'
# tokenizer = tokenization.FullTokenizer(BERT_PATH+'/assets/vocab.txt', True)
tokenizer2 = BertTokenizer.from_pretrained(BERT_PATH+'/assets/vocab.txt', do_lower_case=True,)
MAX_SEQUENCE_LENGTH = 512

df_train = pd.read_csv(PATH+'train.csv',header=0,encoding='utf-8')
df_test = pd.read_csv(PATH+'test.csv',header=0,encoding='utf-8')
df_sub = pd.read_csv(PATH+'sample_submission.csv')
df = pd.concat([df_train,df_test],axis=0,ignore_index=True)
print(f'''Train Shape: {df_train.shape}
Test Shape: {df_test.shape}
Df Shape:{df.shape}''')


output_categories = ['question_asker_intent_understanding', 'question_body_critical', 'question_conversational', 'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer', 'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice', 'question_type_compare', 'question_type_consequence', 'question_type_definition', 'question_type_entity', 'question_type_instructions', 'question_type_procedure', 'question_type_reason_explanation', 'question_type_spelling', 'question_well_written', 'answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance', 'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 'answer_type_reason_explanation', 'answer_well_written']
input_categories = ['question_title', 'question_body', 'answer']
print('\noutput categories:\n\t', output_categories)
print('\ninput categories:\n\t', input_categories)


Train Shape: (6079, 41)
Test Shape: (476, 11)
Df Shape:(6555, 41)

output categories:
	 ['question_asker_intent_understanding', 'question_body_critical', 'question_conversational', 'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer', 'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice', 'question_type_compare', 'question_type_consequence', 'question_type_definition', 'question_type_entity', 'question_type_instructions', 'question_type_procedure', 'question_type_reason_explanation', 'question_type_spelling', 'question_well_written', 'answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance', 'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 'answer_type_reason_explanation', 'answer_well_written']

input categories:
	 ['question_title', 'question_body', 'answer']


In [4]:
for col in output_categories:
    print(f'{col}: {len(df_train[col].unique())}')

question_asker_intent_understanding: 9
question_body_critical: 9
question_conversational: 5
question_expect_short_answer: 5
question_fact_seeking: 5
question_has_commonly_accepted_answer: 5
question_interestingness_others: 9
question_interestingness_self: 9
question_multi_intent: 5
question_not_really_a_question: 5
question_opinion_seeking: 5
question_type_choice: 5
question_type_compare: 5
question_type_consequence: 5
question_type_definition: 5
question_type_entity: 5
question_type_instructions: 5
question_type_procedure: 5
question_type_reason_explanation: 5
question_type_spelling: 3
question_well_written: 9
answer_helpful: 9
answer_level_of_information: 9
answer_plausible: 9
answer_relevance: 9
answer_satisfaction: 17
answer_type_instructions: 5
answer_type_procedure: 5
answer_type_reason_explanation: 5
answer_well_written: 9


In [5]:
df['answer_well_written'].unique()

array([1.        , 0.88888889, 0.77777778, 0.83333333, 0.66666667,
       0.5       , 0.55555556, 0.44444444, 0.33333333,        nan])

In [6]:
sorted(df_train['answer_well_written'].unique())

[0.3333333333333333,
 0.4444444444444444,
 0.5,
 0.5555555555555556,
 0.6666666666666666,
 0.7777777777777778,
 0.8333333333333334,
 0.8888888888888888,
 1.0]

In [7]:
dum_cols = {}
for col in output_categories:
    key = f'{col}'
    dum_cols[key] = pd.get_dummies(df_train[col], drop_first=False, prefix=f'{col}')
    print(f'{col}:{dum_cols[key].shape}')

question_asker_intent_understanding:(6079, 9)
question_body_critical:(6079, 9)
question_conversational:(6079, 5)
question_expect_short_answer:(6079, 5)
question_fact_seeking:(6079, 5)
question_has_commonly_accepted_answer:(6079, 5)
question_interestingness_others:(6079, 9)
question_interestingness_self:(6079, 9)
question_multi_intent:(6079, 5)
question_not_really_a_question:(6079, 5)
question_opinion_seeking:(6079, 5)
question_type_choice:(6079, 5)
question_type_compare:(6079, 5)
question_type_consequence:(6079, 5)
question_type_definition:(6079, 5)
question_type_entity:(6079, 5)
question_type_instructions:(6079, 5)
question_type_procedure:(6079, 5)
question_type_reason_explanation:(6079, 5)
question_type_spelling:(6079, 3)
question_well_written:(6079, 9)
answer_helpful:(6079, 9)
answer_level_of_information:(6079, 9)
answer_plausible:(6079, 9)
answer_relevance:(6079, 9)
answer_satisfaction:(6079, 17)
answer_type_instructions:(6079, 5)
answer_type_procedure:(6079, 5)
answer_type_reason_

In [8]:
def _get_masks(tokens, max_seq_length):
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens)+[0]*(max_seq_length-len(tokens))

def _get_segments(tokens, max_seq_length):
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    
    segments=[]
    first_sep=True
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == '[ANS]':
#             if first_sep:
#                 first_sep=False
#             else:
            current_segment_id=1
    return segments+[0]*(max_seq_length-len(tokens))

def _trim_input(title, question, answer, max_sequence_length, t_max_len=30, q_max_len=239, a_max_len=239):

    t = tokenizer2.tokenize(title)
    q = tokenizer2.tokenize(question)
    a = tokenizer2.tokenize(answer)
    
    t_len = len(t)
    q_len = len(q)
    a_len = len(a)

    if (t_len+q_len+a_len+4) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + floor((t_max_len - t_len)/2)
            q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
        else:
            t_new_len = t_max_len
      
        if a_max_len > a_len:
            a_new_len = a_len 
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
            
            
        if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d" 
                             % (max_sequence_length, (t_new_len+a_new_len+q_new_len+4)))
        
        t = t[:t_new_len]
        q = q[:q_new_len]
        a = a[:a_new_len]
    
    return t, q, a

def _get_ids(tokens, tokenizer, max_seq_length):
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0]*(max_seq_length - len(token_ids))
    return input_ids

def _convert_to_bert_inputs(title, question, answer, tokenizer, max_seq_length):
    stoken = ['[CLS]']+title+['[QBODY]']+question+['[ANS]']+answer+['[SEP]']
    input_ids = _get_ids(tokens=stoken, tokenizer=tokenizer,max_seq_length=max_seq_length)
    input_masks = _get_masks(tokens=stoken, max_seq_length=max_seq_length)
    input_segments = _get_segments(tokens=stoken, max_seq_length=max_seq_length)
    
    return [input_ids, input_masks, input_segments]

def compute_input_array(df, columns, tokenizer, max_sequence_length):
    input_ids, input_masks, input_segments = [], [], []
    for _, col in tqdm(df[columns].iterrows()):
        t, q, a = col['question_title'], col['question_body'], col['answer']
        t,q,a = _trim_input(t,q,a, max_sequence_length)
        ids, masks, segments = _convert_to_bert_inputs(t,q,a, tokenizer, max_sequence_length)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
        
    return [np.array(input_ids, dtype=np.int32), np.array(input_masks, dtype=np.int32), 
            np.array(input_segments, dtype=np.int32)]

def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

In [9]:
def compute_spearmanr(trues, preds):
    rhos=[]
    for t, p in zip(trues.T, preds.T):
        rhos.append(
            spearmanr(t,p).correlation
        )
    return np.nanmean(rhos)

test_predictions=[]

class CustomCallback(keras.callbacks.Callback):
    def __init__(self, val_size,valid_data, test_data, test_predictions=test_predictions, batch_size=16, fold=None):
        self.valid_inputs = valid_data[0]
        self.valid_outputs = valid_data[1]
        self.val_size = val_size
        self.test_inputs = test_data
        self.batch_size = batch_size
        self.test_predictions = test_predictions
        self.fold = fold
        
    def on_train_begin(self, logs={}):
        self.valid_predictions=[]
        self.valid_preds = np.zeros((self.val_size, len(output_categories)), dtype=np.float64)
        self.test_preds = np.zeros((len(self.test_inputs[0]), len(output_categories)), dtype=np.float64)
        #self.test_predictions=[]
        
    def on_epoch_end(self, epoch, logs={}):
        out=self.model.predict(self.valid_inputs, batch_size=self.batch_size)
        
#         valid_preds = np.zeros((self.val_size, len(output_categories)), dtype=np.float64)
        for i,col in enumerate(output_categories):
            for j in range(0,self.val_size):
                self.valid_preds[j][i]=sorted(df_train[col].unique())[np.argmax(out[i][j])]
        
        self.valid_predictions.append(self.valid_preds)
        
        
        
        rho_val = compute_spearmanr(self.valid_outputs, np.average(self.valid_predictions, axis=0))
        print(f"\nvalidation rho: {round(rho_val,4)}")
#         test_preds
        test_out=self.model.predict(self.test_inputs, batch_size=self.batch_size)
        for i,col in enumerate(output_categories):
            for j in range(len(self.test_inputs[0])):
                self.test_preds[j][i]=sorted(df_train[col].unique())[np.argmax(out[i][j])]
        
        self.test_predictions.append(self.test_preds)
        
#         self.model.save_weights(f'/kaggle/working/bert-base-{fold}-{epoch}.hdf5')
            
        


In [10]:
# bert_config=BertConfig(unk_token="[QBODY]", pad_token="[ANS]").from_pretrained('../input/bert-tensorflow/bert-base-uncased-config.json',output_hidden_states=True, num_labels=30)
# def bertModel():
#     input_ids = keras.layers.Input((MAX_SEQUENCE_LENGTH), dtype = tf.int32, name = 'input_word_ids')
#     input_mask = keras.layers.Input((MAX_SEQUENCE_LENGTH), dtype = tf.int32, name = 'input_masks')
#     input_segments = keras.layers.Input((MAX_SEQUENCE_LENGTH), dtype = tf.int32, name = 'input_segments')
    
#     bert_model = TFBertModel.from_pretrained(pretrained_model_name_or_path='../input/bert-tensorflow/bert-base-uncased-tf_model.h5',config = bert_config)
# #     bert_model.resize_token_embeddings(len(tokenizer2))
#     sequence_output, pooler_output, hidden_states = bert_model([input_ids,input_mask, input_segments])
    
#     h12 = tf.reshape(hidden_states[-1][:,0],(-1,1,768))
#     h11 = tf.reshape(hidden_states[-2][:,0],(-1,1,768))
#     h10 = tf.reshape(hidden_states[-3][:,0],(-1,1,768))
#     h09 = tf.reshape(hidden_states[-4][:,0],(-1,1,768))
#     concat_hidden = keras.layers.Concatenate(axis=2)([h12, h11, h10, h09])
# #     concat_hidden = tf.reshape(concat_hidden, (-1,768*4))
#     x = keras.layers.GlobalAveragePooling1D()(concat_hidden)
# #     dense1 = keras.layers.Dense(768)(x)
# #     dense1 = keras.layers.LeakyReLU()(dense1)
# #     x = keras.layers.Add()([dense1,x])    
#     x = keras.layers.Dropout(0.2)(x)
#     out = keras.layers.Dense(len(output_categories), activation='sigmoid', name='final_dense_output')(x)
    
#     model = keras.models.Model(inputs=[input_ids, input_mask, input_segments], outputs=out)
#     model.compile(loss='binary_crossentropy', optimizer = keras.optimizers.Adam(lr=3e-5))
#     return model

In [11]:
bert_config=BertConfig(unk_token="[QBODY]", pad_token="[ANS]").from_pretrained('../input/bert-tensorflow/bert-base-uncased-config.json',output_hidden_states=True, num_labels=30)

def dense_layer(bert_in,units, name, activation='softmax'):
    d = keras.layers.Dense(units, activation=activation, name=name)(bert_in)
    return d

def bertModel(output_categories=output_categories, dum_cols = dum_cols):
    input_ids = keras.layers.Input((MAX_SEQUENCE_LENGTH), dtype = tf.int32, name = 'input_word_ids')
    input_mask = keras.layers.Input((MAX_SEQUENCE_LENGTH), dtype = tf.int32, name = 'input_masks')
    input_segments = keras.layers.Input((MAX_SEQUENCE_LENGTH), dtype = tf.int32, name = 'input_segments')
    
    bert_model = TFBertForSequenceClassification.from_pretrained(pretrained_model_name_or_path='../input/bert-tensorflow/bert-base-uncased-tf_model.h5',config = bert_config)
#     bert_model.resize_token_embeddings(len(tokenizer2))
    logits, hidden_states = bert_model([input_ids,input_mask, input_segments])
    
    h12 = tf.reshape(hidden_states[-1][:,0],(-1,1,768))
    h11 = tf.reshape(hidden_states[-2][:,0],(-1,1,768))
    h10 = tf.reshape(hidden_states[-3][:,0],(-1,1,768))
    h09 = tf.reshape(hidden_states[-4][:,0],(-1,1,768))
    concat_hidden = keras.layers.Concatenate(axis=2)([h12, h11, h10, h09])
#     concat_hidden = tf.reshape(concat_hidden, (-1,768*4))
    x = keras.layers.GlobalAveragePooling1D()(concat_hidden)
# #     dense1 = keras.layers.Dense(768)(x)
# #     dense1 = keras.layers.LeakyReLU()(dense1)
# #     x = keras.layers.Add()([dense1,x])    
#     x = keras.layers.Dropout(0.2)(x)
    out = {}
    for col in output_categories:
        key=f'{col}'
        out[key]=dense_layer(bert_in=x,units=dum_cols[key].shape[1], name=f'out_{col}', activation='softmax')
    
    model = keras.models.Model(inputs=[input_ids, input_mask, input_segments], 
                               outputs=[out['question_asker_intent_understanding'],
                                        out['question_body_critical'],
                                        out['question_conversational'],
                                        out['question_expect_short_answer'],
                                        out['question_fact_seeking'],
                                        out['question_has_commonly_accepted_answer'],
                                        out['question_interestingness_others'],
                                        out['question_interestingness_self'],
                                        out['question_multi_intent'],
                                        out['question_not_really_a_question'],
                                        out['question_opinion_seeking'],
                                        out['question_type_choice'],
                                        out['question_type_compare'],
                                        out['question_type_consequence'],
                                        out['question_type_definition'],
                                        out['question_type_entity'],
                                        out['question_type_instructions'],
                                        out['question_type_procedure'],
                                        out['question_type_reason_explanation'],
                                        out['question_type_spelling'],
                                        out['question_well_written'],
                                        out['answer_helpful'],
                                        out['answer_level_of_information'],
                                        out['answer_plausible'],
                                        out['answer_relevance'],
                                        out['answer_satisfaction'],
                                        out['answer_type_instructions'],
                                        out['answer_type_procedure'],
                                        out['answer_type_reason_explanation'],
                                        out['answer_well_written']])
                               
    model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(lr=3e-5), metrics=['accuracy'])
    return model

In [12]:
# model = bertModel()
# model.summary()

In [13]:
df_train.columns

Index(['qa_id', 'question_title', 'question_body', 'question_user_name',
       'question_user_page', 'answer', 'answer_user_name', 'answer_user_page',
       'url', 'category', 'host', 'question_asker_intent_understanding',
       'question_body_critical', 'question_conversational',
       'question_expect_short_answer', 'question_fact_seeking',
       'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfa

In [14]:
# gkf = GroupKFold(n_splits=10).split(X=df_train.question_body, groups=df_train.question_body)

outputs = compute_output_arrays(df_train, output_categories)
inputs = compute_input_array(df_train, input_categories, tokenizer2, MAX_SEQUENCE_LENGTH)
test_inputs = compute_input_array(df_test, input_categories, tokenizer2, MAX_SEQUENCE_LENGTH)

6079it [01:19, 76.54it/s]
476it [00:06, 74.53it/s]


In [15]:
gkf = GroupKFold(n_splits=10).split(X=df_train.question_body, groups=df_train.question_body)
# train, valid, y_train, y_val = train_test_split(inputs, outputs, test_size=0.15, stratify=df_train['category'])

In [16]:
# H = model.fit(inputs, [dum_cols['question_asker_intent_understanding'],
#  dum_cols['question_body_critical'],
#  dum_cols['question_conversational'],
#  dum_cols['question_expect_short_answer'],
#  dum_cols['question_fact_seeking'],
#  dum_cols['question_has_commonly_accepted_answer'],
#  dum_cols['question_interestingness_others'],
#  dum_cols['question_interestingness_self'],
#  dum_cols['question_multi_intent'],
#  dum_cols['question_not_really_a_question'],
#  dum_cols['question_opinion_seeking'],
#  dum_cols['question_type_choice'],
#  dum_cols['question_type_compare'],
#  dum_cols['question_type_consequence'],
#  dum_cols['question_type_definition'],
#  dum_cols['question_type_entity'],
#  dum_cols['question_type_instructions'],
#  dum_cols['question_type_procedure'],
#  dum_cols['question_type_reason_explanation'],
#  dum_cols['question_type_spelling'],
#  dum_cols['question_well_written'],
#  dum_cols['answer_helpful'],
#  dum_cols['answer_level_of_information'],
#  dum_cols['answer_plausible'],
#  dum_cols['answer_relevance'],
#  dum_cols['answer_satisfaction'],
#  dum_cols['answer_type_instructions'],
#  dum_cols['answer_type_procedure'],
#  dum_cols['answer_type_reason_explanation'],
#  dum_cols['answer_well_written']], batch_size=8, epochs=2, validation_split=0.15, verbose=1)

In [17]:
histories = []

for fold, (train_idx, valid_idx) in enumerate(gkf):
#     print(len(train_idx), len(valid_idx))
    if fold<3:
        keras.backend.clear_session()
        model = bertModel()
        
        train_inputs = [inputs[i][train_idx] for i in range(3)]
        train_outputs = outputs[train_idx]

        valid_inputs = [inputs[i][valid_idx] for i in range(3)]
        valid_outputs = outputs[valid_idx]
    
        custom_callback = CustomCallback(val_size=len(valid_idx),valid_data=(valid_inputs, valid_outputs), test_data=test_inputs,
                                         batch_size=8, fold=fold)
        H = model.fit(train_inputs, [dum_cols['question_asker_intent_understanding'].iloc[train_idx,:].values,
 dum_cols['question_body_critical'].iloc[train_idx,:].values,
 dum_cols['question_conversational'].iloc[train_idx,:].values,
 dum_cols['question_expect_short_answer'].iloc[train_idx,:].values,
 dum_cols['question_fact_seeking'].iloc[train_idx,:].values,
 dum_cols['question_has_commonly_accepted_answer'].iloc[train_idx,:].values,
 dum_cols['question_interestingness_others'].iloc[train_idx,:].values,
 dum_cols['question_interestingness_self'].iloc[train_idx,:].values,
 dum_cols['question_multi_intent'].iloc[train_idx,:].values,
 dum_cols['question_not_really_a_question'].iloc[train_idx,:].values,
 dum_cols['question_opinion_seeking'].iloc[train_idx,:].values,
 dum_cols['question_type_choice'].iloc[train_idx,:].values,
 dum_cols['question_type_compare'].iloc[train_idx,:].values,
 dum_cols['question_type_consequence'].iloc[train_idx,:].values,
 dum_cols['question_type_definition'].iloc[train_idx,:].values,
 dum_cols['question_type_entity'].iloc[train_idx,:].values,
 dum_cols['question_type_instructions'].iloc[train_idx,:].values,
 dum_cols['question_type_procedure'].iloc[train_idx,:].values,
 dum_cols['question_type_reason_explanation'].iloc[train_idx,:].values,
 dum_cols['question_type_spelling'].iloc[train_idx,:].values,
 dum_cols['question_well_written'].iloc[train_idx,:].values,
 dum_cols['answer_helpful'].iloc[train_idx,:].values,
 dum_cols['answer_level_of_information'].iloc[train_idx,:].values,
 dum_cols['answer_plausible'].iloc[train_idx,:].values,
 dum_cols['answer_relevance'].iloc[train_idx,:].values,
 dum_cols['answer_satisfaction'].iloc[train_idx,:].values,
 dum_cols['answer_type_instructions'].iloc[train_idx,:].values,
 dum_cols['answer_type_procedure'].iloc[train_idx,:].values,
 dum_cols['answer_type_reason_explanation'].iloc[train_idx,:].values,
 dum_cols['answer_well_written'].iloc[train_idx,:].values], batch_size=8, epochs=4, verbose=1, callbacks=[custom_callback])
        histories.append(H)        

    

Train on 5471 samples
Epoch 1/4
5464/5471 [============================>.] - ETA: 0s - loss: 31.5488 - out_question_asker_intent_understanding_loss: 1.4043 - out_question_body_critical_loss: 1.9169 - out_question_conversational_loss: 0.4044 - out_question_expect_short_answer_loss: 1.3659 - out_question_fact_seeking_loss: 1.1197 - out_question_has_commonly_accepted_answer_loss: 0.9997 - out_question_interestingness_others_loss: 1.6959 - out_question_interestingness_self_loss: 1.6798 - out_question_multi_intent_loss: 1.1369 - out_question_not_really_a_question_loss: 0.0752 - out_question_opinion_seeking_loss: 1.4171 - out_question_type_choice_loss: 1.1389 - out_question_type_compare_loss: 0.3179 - out_question_type_consequence_loss: 0.1256 - out_question_type_definition_loss: 0.2391 - out_question_type_entity_loss: 0.4516 - out_question_type_instructions_loss: 1.1028 - out_question_type_procedure_loss: 0.9953 - out_question_type_reason_explanation_loss: 1.2936 - out_question_type_spellin

In [18]:
#sddzdfdvhfdgaggeggbsrdvsddfassfstrtryjymrtygcc

In [19]:
print(len(test_predictions))

12


In [20]:
test_preds = [test_predictions[i] for i in range(len(test_predictions))]
test_preds = [np.average(test_preds[i], axis=0) for i in range(len(test_preds))]
test_preds = np.mean(test_predictions, axis=0)

df_sub.iloc[:, 1:] = test_preds

df_sub.to_csv('submission.csv', index=False)

In [21]:
# del model
# gc.collect()

In [22]:
# zz = BertConfig.from_pretrained('../input/bert-tensorflow/bert-base-uncased-config.json',output_hidden_states=True)
# class TFBertPreTrainedModel(TFPreTrainedModel):
#     """ An abstract class to handle weights initialization and
#         a simple interface for dowloading and loading pretrained models.
#     """

#     config_class = zz
#     pretrained_model_archive_map = {"bert-base-uncased": "../input/bert-tensorflow/bert-base-uncased-tf_model.h5"}
#     base_model_prefix = "bert"


# class TFBert_v2(TFBertPreTrainedModel):
#     def __init__(self, config='../input/bert-tensorflow/bert-base-uncased-config.json', *inputs, **kwargs):
#         self.config = config
#         self.TF_BERT_PRETRAINED_MODEL_ARCHIVE_MAP = {"bert-base-uncased": "../input/bert-tensorflow/bert-base-uncased-tf_model.h5"}
#         super(TFBert_v2, self).__init__(self.config, *inputs, **kwargs)
#         self.bert = TFBertMainLayer(config, name="bert")

#     def call(self, inputs, **kwargs):
#         outputs = self.bert(inputs, **kwargs)
#         return outputs

In [23]:
# class TFBertForSequenceClassification(TFBertPreTrainedModel):
#     r"""
#     Outputs: `Tuple` comprising various elements depending on the configuration (config) and inputs:
#         **logits**: ``Numpy array`` or ``tf.Tensor`` of shape ``(batch_size, config.num_labels)``
#             Classification (or regression if config.num_labels==1) scores (before SoftMax).
#         **hidden_states**: (`optional`, returned when ``config.output_hidden_states=True``)
#             list of ``Numpy array`` or ``tf.Tensor`` (one for the output of each layer + the output of the embeddings)
#             of shape ``(batch_size, sequence_length, hidden_size)``:
#             Hidden-states of the model at the output of each layer plus the initial embedding outputs.
#         **attentions**: (`optional`, returned when ``config.output_attentions=True``)
#             list of ``Numpy array`` or ``tf.Tensor`` (one for each layer) of shape ``(batch_size, num_heads, sequence_length, sequence_length)``:
#             Attentions weights after the attention softmax, used to compute the weighted average in the self-attention heads.

#     Examples::

#         import tensorflow as tf
#         from transformers import BertTokenizer, TFBertForSequenceClassification

#         tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#         model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
#         input_ids = tf.constant(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True))[None, :]  # Batch size 1
#         outputs = model(input_ids)
#         logits = outputs[0]

#     """

#     def __init__(self, config, *inputs, **kwargs):
#         super(TFBertForSequenceClassification, self).__init__(config, *inputs, **kwargs)
#         self.num_labels = config.num_labels

#         self.bert = TFBertMainLayer(config, name="bert")
#         self.dropout = tf.keras.layers.Dropout(config.hidden_dropout_prob)
#         self.classifier = tf.keras.layers.Dense(
#             config.num_labels, kernel_initializer=get_initializer(config.initializer_range), name="classifier"
#         )

#     def call(self, inputs, **kwargs):
#         outputs = self.bert(inputs, **kwargs)

#         pooled_output = outputs[1]

#         pooled_output = self.dropout(pooled_output, training=kwargs.get("training", False))
#         logits = self.classifier(pooled_output)

#         outputs = (logits,) + outputs[1:]  # add pooled_output, hidden states and attention if they are here

#         return outputs

In [24]:
# bert_config=BertConfig(unk_token="[QBODY]", pad_token="[ANS]").from_pretrained('../input/bert-tensorflow/bert-base-uncased-config.json',output_hidden_states=True, num_labels=1)
# m = TFBertForSequenceClassification.from_pretrained(pretrained_model_name_or_path='../input/bert-tensorflow/bert-base-uncased-tf_model.h5', config = bert_config)

In [25]:
# bert_config.

In [26]:
# bert_config2=BertConfig(unk_token="[QBODY]", pad_token="[ANS]").from_pretrained('../input/bert-tensorflow/bert-base-uncased-config.json',output_hidden_states=True)
# m2 = TFBertModel.from_pretrained(pretrained_model_name_or_path='../input/bert-tensorflow/bert-base-uncased-tf_model.h5', config = bert_config2)

In [27]:

# input_ids=tf.constant(tokenizer2.encode(text="Hello, my dog is cute", add_special_tokens=True))[None, :]  # Batch size 1
# logits,pool, hidden_states = m(input_ids)

In [28]:

# input_ids=tf.constant(tokenizer2.encode(text="Hello, my dog is cute", add_special_tokens=True))[None, :]  # Batch size 1
# sequence_output, pooler_output, hidden_states2 = m2(input_ids)